In [ ]:
# %%capture
# !pip install git+https://github.com/biothings/biothings_explorer#egg=biothings_explorer

In [1]:
import os
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
from datetime import date
import datetime as dt
import pandas as pd
import sys
import csv
import numpy as np
import multiprocessing

In [2]:
import psycopg2
import pandas.io.sql as sqlio
# from tabulate import tabulate

In [3]:
import json
import argparse
import lxml
import config

# from Authentication import *
import requests
import json
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry


In [ ]:
url = "https://aact.ctti-clinicaltrials.org/pipe_files"
response = requests.get(url)

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.text)
# print(soup.prettify())

In [ ]:

# https://aact.ctti-clinicaltrials.org/static/exported_files/monthly/20220301_pipe-delimited-export.zip
# https://aact.ctti-clinicaltrials.org/static/exported_files/monthly/20220801_pipe-delimited-export.zip
upload_dates = []
zip_files = []
links = []
body = soup.find_all('td', attrs={'class': 'file-archive'}) #Find all
for el in body:
    tags = el.find('a')
    try:
        if 'href' in tags.attrs:   # looking for href inside anchor tag    
            link = "https://aact.ctti-clinicaltrials.org" + tags.get('href')
            links.append(link)
            last_upload = link.split("/")[-1]
            zip_files.append(last_upload)
            date_upload = last_upload.split("_")[0]
            upload_dates.append(date_upload)    # appending link to list of links
    except:    # pass if list missing anchor tag or anchor tag does not has a href params 
        pass
    
print(upload_dates)

upload_dates = [dt.datetime.strptime(date, '%Y%m%d').date() for date in upload_dates] # convert all strings in list into datetime objects


In [ ]:
most_recent_date = max(upload_dates)
# print(most_recent_date)

today = dt.date.today()

# UNCOMMENT LINE BELOW WHEN IN PRODUCTION, THIS IS FOR TESTING PURPOSES
most_recent_date = min(upload_dates)

# fix this to run entire script should the current date be 
if most_recent_date < today:
#     date_file_df = pd.DataFrame(list(zip(upload_dates, zip_files, links)))
#     print(links[0])
    resp = urlopen(links[0])
else:
    print("Local instance of clinical trials data is already up to date.")



In [ ]:
pd.set_option('display.max_colwidth', 3000)
# date_file_df[2]

In [ ]:
os.getcwd()


In [4]:
# connect to DB and get the column names of the table
con = None
con = psycopg2.connect(database="aact")
con.rollback()
cursor = con.cursor()

con.autocommit = True # SQL statement is treated as a transaction and is automatically committed right after it is executed
# grab the conditions
sql = '''SELECT * FROM ctgov.conditions;'''
cursor.execute(sql)
column_names = [desc[0] for desc in cursor.description]
tuples = cursor.fetchall()
conditions_df = pd.DataFrame(tuples, columns=column_names)

#grab the browse_conditions
sql = '''SELECT * FROM ctgov.browse_conditions;'''
cursor.execute(sql)
column_names = [desc[0] for desc in cursor.description]
tuples = cursor.fetchall()
browse_conditions_df = pd.DataFrame(tuples, columns=column_names)

#grab the interventions
sql = '''SELECT * FROM ctgov.interventions;'''
cursor.execute(sql)
column_names = [desc[0] for desc in cursor.description]
tuples = cursor.fetchall()
interventions_df = pd.DataFrame(tuples, columns=column_names)

#grab the browse_interventions
sql = '''SELECT * FROM ctgov.browse_interventions;'''
cursor.execute(sql)
column_names = [desc[0] for desc in cursor.description]
tuples = cursor.fetchall()
browse_interventions_df = pd.DataFrame(tuples, columns=column_names)

con.close()


In [5]:
interventions_df = interventions_df.rename(columns={'id': 'int_id',
                                                    'nct_id': 'int_nctid',
                                                    'intervention_type': 'int_type',
                                                    'name': 'int_name',
                                                    'description': 'int_description'})
interventions_df

,int_id,int_nctid,int_type,int_name,int_description
0,4547568,NCT01340339,Device,Phototherapy,Fluorescent reverse phototherapy (7 white ligh...
1,4547569,NCT01340339,Device,Phototherapy,super LED reverse phototherapy (17 bulbs arran...
2,4547570,NCT01340365,Behavioral,Tai Chi Exercise,Practicing Tai Chi exercise 4 times a week for...
3,4547571,NCT01340365,Behavioral,Tai Chi,"Usual care, individuals attend testing session..."
4,4547572,NCT01340391,Device,Omnicast,dorsal splint 2-5 weeks
...,...,...,...,...,...
685271,4139681,NCT03104712,Other,Rice + Pesto,50g available carbohydrate
685272,4139682,NCT03104465,Behavioral,Mindfulness Training,interactive. web-based mindfulness training co...
685273,4139683,NCT03104361,Biological,Platelet-Rich Plasma,Patients who meet all eligible requirements fo...
685274,4139684,NCT03104283,Drug,Apatinib,take apatinib orally until disease progression...


In [6]:
interventions_df = interventions_df.drop(columns=['int_id', 'int_description'])

In [7]:
conditions_df = conditions_df.rename(columns={'id': 'con_id',
                                              'nct_id': 'con_nctid',
                                              'name': 'con_name',
                                              'downcase_name': 'con_downcase_name'})
conditions_df

,con_id,con_nctid,con_name,con_downcase_name
0,3072834,NCT03254264,Autism Spectrum Disorder,autism spectrum disorder
1,3072835,NCT03254329,Human Milk Nutrient Reference Values,human milk nutrient reference values
2,3349485,NCT03460652,ADHD,adhd
3,3349486,NCT03460899,Diabetes Mellitus With Hypoglycemia,diabetes mellitus with hypoglycemia
4,3072836,NCT03254342,Major Depressive Disorder,major depressive disorder
...,...,...,...,...
688783,3072052,NCT03261687,Pregnancy Related,pregnancy related
688784,3072053,NCT03261687,Pelvic Girdle Pain,pelvic girdle pain
688785,3072054,NCT03261622,Faecal Incontinence,faecal incontinence
688786,3072055,NCT03261622,Fecal Incontinence,fecal incontinence


In [8]:
conditions_df = conditions_df.drop(columns=['con_id', 'con_name'])

In [9]:
browse_interventions_df = browse_interventions_df.rename(columns={'id': 'browseint_id',
                                                                  'nct_id': 'browseint_nctid',
                                                                  'mesh_term': 'browseint_meshterm',
                                                                  'downcase_mesh_term': 'browseint_meshterm_downcase',
                                                                  'mesh_type': 'browseint_meshtype'})
browse_interventions_df

,browseint_id,browseint_nctid,browseint_meshterm,browseint_meshterm_downcase,browseint_meshtype
0,4935968,NCT04327843,Haloperidol,haloperidol,mesh-list
1,4935969,NCT04327843,Haloperidol decanoate,haloperidol decanoate,mesh-list
2,4935970,NCT04327843,Antiemetics,antiemetics,mesh-ancestor
3,5198960,NCT00808223,Alefacept,alefacept,mesh-list
4,5198961,NCT00808223,Dermatologic Agents,dermatologic agents,mesh-ancestor
...,...,...,...,...,...
1335262,4400668,NCT03100786,"Antineoplastic Agents, Hormonal","antineoplastic agents, hormonal",mesh-ancestor
1335263,4400669,NCT03100786,Antineoplastic Agents,antineoplastic agents,mesh-ancestor
1335264,4400670,NCT03100565,Lidocaine,lidocaine,mesh-list
1335265,4400671,NCT03100565,"Anesthetics, Local","anesthetics, local",mesh-ancestor


In [10]:
browse_interventions_df = browse_interventions_df.drop(columns=['browseint_id', 'browseint_meshterm'])

In [11]:
browse_conditions_df = browse_conditions_df.rename(columns={'id': 'browsecon_id',
                                                            'nct_id': 'browsecon_nctid',
                                                            'mesh_term': 'browsecon_meshterm',
                                                            'downcase_mesh_term': 'browsecon_meshterm_downcase',
                                                            'mesh_type': 'browsecon_meshtype'})
browse_conditions_df

,browsecon_id,browsecon_nctid,browsecon_meshterm,browsecon_meshterm_downcase,browsecon_meshtype
0,9631397,NCT04035564,Hyponatremia,hyponatremia,mesh-list
1,9631398,NCT04035564,Water-Electrolyte Imbalance,water-electrolyte imbalance,mesh-ancestor
2,9631399,NCT04035564,Metabolic Diseases,metabolic diseases,mesh-ancestor
3,9631400,NCT04032652,Colitis,colitis,mesh-list
4,9631401,NCT04032652,"Colitis, Ulcerative","colitis, ulcerative",mesh-list
...,...,...,...,...,...
2573556,8601072,NCT03213899,Disease Attributes,disease attributes,mesh-ancestor
2573557,8601073,NCT03213899,Pathologic Processes,pathologic processes,mesh-ancestor
2573558,8601074,NCT03214016,Hypertension,hypertension,mesh-list
2573559,8601075,NCT03214016,Vascular Diseases,vascular diseases,mesh-ancestor


In [12]:
browse_conditions_df = browse_conditions_df.drop(columns=['browsecon_id', 'browsecon_meshterm'])

In [13]:
print(len(interventions_df['int_name'].unique()))
print(len(conditions_df['con_downcase_name'].unique()))
print(len(browse_conditions_df['browsecon_meshterm_downcase'].unique()))
print(len(browse_interventions_df['browseint_meshterm_downcase'].unique()))

350361
95250
4280
3744


In [ ]:
conditions_df


In [14]:
df = pd.merge(conditions_df, interventions_df, left_on='con_nctid', right_on = 'int_nctid')

In [ ]:
df.shape

In [15]:
df_dedup = df.drop_duplicates(subset = ['con_downcase_name', 'int_name'],
                                      keep = 'first').reset_index(drop = True)

In [ ]:
df_dedup.shape

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df_dedup.head(1000))

In [ ]:
df_dedup[['con_downcase_name', 'int_name']].to_csv('ClinTrials_KG_edges.csv', sep ='\t', index=False)


# notes on creating edge file

go to /proj/arivale/gglusman/KGs

once there look into wellness/source/wellness_kg_edges_v1.6.tsv

notice the first line: subject	predicate	object	relation	subject_name	object_name	category	N	Type_of_relationship	Strength_of_relationship	qualifiers	qualifier_value	Bonferroni_pval

subject, predicate, object are required (maybe some others, don’t remember)

…and they should be CURIEs

once we map concepts to biolink, mesh etc we’ll have curies

meanwhile, the content you have fits subject_name, object_name

the predicate is what we made up… let’s say ‘evaluated_for’

for short

we’ll have to find a suitable biolink predicate, or suggest a new one

so interventions would be subjects, and conditions would be objects

looking at the several first lines in your file, I see some that make sense, others that seem to suggest there are data problems

for example, “metastatic lung cancer	Pathways” -> Pathways doesn’t seem to be an intervention

‘placebo’ we’ll have to suppress

make sure to include the NCT id in a (non-required) column

it’s essentially part of the provenance

we’ll want to add multiple additional columns with things like FDA approval status, sample size, etc etc etc

look also into EHR/source/ehr_risk_kg_edges_2021_05_07.tsv

…and of course replace ‘edges’ -> ‘nodes’ for their cognate nodes files

wellness/source/wellness_kg_edges_v1.6.tsv -> wellness/source/wellness_kg_nodes_v1.6.tsv

drop "control"

‘control’ is clear, ‘control group’ too… but ‘Control Ostomy Barrier’ is likely not a ‘control’


# My own notes

Use pool/multiprocess to make faster
resolve multiple possible CURIEs per concept

In [16]:
# first get only relevant columns from DB
ct_extract = pd.DataFrame(df_dedup[['con_nctid', 'con_downcase_name', 'int_type', 'int_name']])
ct_extract = ct_extract.rename(columns={'con_nctid': 'nctid'})
# get CURIE column for nct_id column (https://bioregistry.io/registry/clinicaltrials)

ct_extract['nctid_curie'] = ct_extract['nctid']
ct_extract['nctid_curie'] = 'clinicaltrials:' + ct_extract['nctid'].astype(str)

In [ ]:
ct_extract

In [ ]:
# let's map all the conditions to BTE
# NOTE: we are attempting to match to BTE terms, but many of the conditions terms are not standardized and are misspelled
# First map as many terms as possible to BTE. Whatever is left over, will use other ontologies to create CURIEs


In [ ]:
# recommedation to first format the output as close to sample as possible

In [17]:
final_df = pd.DataFrame(columns=['subject','predicate','object','relation','subject_name','object_name','category'])

In [18]:
# no_col_names_df.columns = col_names_df.columns
# chosen predicate = "related_to", bc "associated_with" implies a statistical relationship 
final_df['subject'] = 'condition:' + ct_extract['con_downcase_name'].astype(str)
final_df['predicate'] = 'biolink:related_to biolink:clinically_demonstrated_to biolink: hypothesized_to'
final_df['object'] = 'intervention:' + ct_extract['int_name'].astype(str)   # this will not all be RxNorm CURIEs since some interventions are not drugs
final_df['relation'] = 'RO:????'
final_df.subject_name = ct_extract.con_downcase_name
final_df.object_name = ct_extract.int_name
final_df.category = 'biolink:Association'
final_df['nctid'] = ct_extract['nctid']
final_df['nctid_curie'] = ct_extract['nctid_curie']


In [ ]:
final_df

In [ ]:
psoriasis = final_df[final_df['subject_name'].str.contains('psoriasis')]
psoriasis



In [ ]:
zinc = final_df[final_df['object_name'].str.contains('zinc', na=False)]
zinc

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
zinc

In [ ]:
psoriasis[['subject_name', 'object_name', 'nctid']].to_csv('psoriasis.csv', sep =',', index=False)
# zinc[['subject_name', 'object_name', 'nctid']].to_csv('zinc.csv', sep ='\t', index=False)


In [ ]:
psoriasis_zinc = psoriasis[psoriasis['object_name'].str.contains('zn')]

In [ ]:
psoriasis_zinc

In [ ]:
final_df.to_csv('ClinTrials_KG_edges.csv', sep ='\t', index=False)


In [ ]:
# interventions will have to be mapped to more than 1 vocabulary unlike conditions, which seems more easily mapped to SNOMED
# first look at the types of interventions (unique on int_type in ct_extract dataframe)

In [19]:
# adding Folder_2/subfolder to the system path
sys.path.insert(0, '/Volumes/TOSHIBA_EXT/ISB/clinical_trials/pymetamap-master')
  
from pymetamap import MetaMap

# NOTE: YOU MUST START the SKR/Medpost Part-of-Speech Tagger Server and the Word Sense Disambiguation (WSD) Server IN TERMINAL

# % ./bin/skrmedpostctl start
# % ./bin/wsdserverctl start


# stop these servers afterwards



In [ ]:
# get the directory of the the module
print(os.getcwd())
# Path
path = "/Volumes/TOSHIBA_EXT/ISB/clinical_trials/public_mm/bin"  
# Path of Start directory
start = os.getcwd()
print(os.path.relpath("/Volumes/TOSHIBA_EXT/ISB/clinical_trials/public_mm/bin", os.getcwd()))

In [20]:
mm = MetaMap.get_instance('/Volumes/TOSHIBA_EXT/ISB/clinical_trials/public_mm/bin/metamap18')

In [79]:
sents = ['myocardїal infarction']  # this has to be 1 sentence
concepts,error = mm.extract_concepts(sents)
for concept in concepts:
    print(concept)
    print("\n")

In [82]:
error

In [ ]:
# check char encoding, is ascii??
    
for ct_string in final_df['subject_name'][:40]:
#     print(type(ct_string))
    print(ct_string.isascii())
    

In [83]:
# MetaMap does not work on non-ascii char, so create dict with what non-ascii char should be replaced with

# diseases = list(set(list(final_df['subject_name'])))
ascii_char_dict = {",": ' ',
                   ';': ' ',
                   '；': ' ',
                   '≥': ' greater than or equal to ',
                   '≤': ' less than or equal to ',
                   '<': ' less than or equal to ',
                   '>': ' less than or equal to ',
                   'ç': 'c',
                   '®': ' ',
                   '©': ' ',
                   'ï': 'i',
                   'ї': 'i',
                   '̇': '',
                   '"̇': ' ',
#                    '"̇"': ' ',
                   '"': ' ',
                   '\"': ' ',
                   'é': 'e',
                   'è': 'e',
                   'í': 'i',
                   '₁': '1',
                   'ı': '1',
                   '，': ' ',
                   'β': ' beta ',
                   'ß': ' beta ',
                   'ö': 'o',
                   'ō': 'o',
                   '´': "",
                   '²': '2',
                   'γ': ' gamma ',
                   'α': ' alpha ',
                   'ü': 'u',
                   'μg': 'micrograms',
                   'µg': 'micrograms',
                   'µ': 'mu',
                   'μ': 'mu',
                   '•': ' ',
                   '°': ' degrees ',
                   'λ': ' lambda ',
                   'ﬂ': 'fl',
                   'ⅱ': '2',
                   'δ': ' delta ',
#                    'stage i': 'stage 1',
#                    'stage ii': 'stage 2',
#                    'stage iii': 'stage 3',
                   'ⅲ': '3',
                   '（': ' ',
                   '）': ' ',
                   '、': ' ',
                   '¨': ' ',
                   '™': ' ',
                   '：': ' ',
                   'œ': ' ',
                   '／': ' ',
                   '.': ' ',
                   '±': ' ',
                   'ő': 'o'
                   
                  }


In [84]:
diseases = list(set(list(final_df['subject_name'])))
interventions = list(filter(None, final_df['object_name'].unique()))

In [143]:
non_ascii = "[^\x00-\x7F]"
pattern = re.compile(r"[^\x00-\x7F]")

# mod_string = re.sub(pattern, '', org_string )

import re

def custom_make_translation(text, translation):
    non_ascii_text = re.sub(pattern, ' ', text)

    return non_ascii_text


text = "split-virion, non-adjuvanted H1N1 vaccine of 15 μg"
translated = custom_make_translation(text, ascii_char_dict)



In [144]:
translated


'split-virion, non-adjuvanted H1N1 vaccine of 15  g'

In [85]:
# test = final_df.apply(lambda row: custom_make_translation(row['subject_name'], ascii_char_dict), axis=1)
# test.to_csv('test_ascii_stripper.csv', sep ='\t', index=False)

import re

def custom_make_translation(text, translation):
    regex = re.compile('|'.join(map(re.escape, translation)))
    regexed_text = regex.sub(lambda match: translation[match.group(0)], text)
    translated_text = regexed_text.encode('ascii', 'replace')
    return translated_text

text = "split-virion, non-adjuvanted H1N1 vaccine of 15 μg"
translated = custom_make_translation(text, ascii_char_dict)


In [86]:
diseases_translated = dict((i, custom_make_translation(i, ascii_char_dict)) for i in diseases)
interventions_translated = dict((i, custom_make_translation(i, ascii_char_dict)) for i in interventions)

In [98]:
with open("diseases.txt", 'w') as f: 
    for value in diseases_translated.values(): 
        f.write('%s\n' % (value))
        
with open("interventions.txt", 'w') as f: 
    for value in interventions_translated.values(): 
        f.write('%s\n' % (value))

In [99]:
from __future__ import print_function
import io

with io.open('diseases.txt','r',encoding='utf-8',errors='ignore') as infile, \
     io.open('diseases_ascii.txt','w',encoding='ascii',errors='ignore') as outfile:
    for line in infile:
        print(*line.split(), file=outfile)
with io.open('interventions.txt','r',encoding='utf-8',errors='ignore') as infile, \
     io.open('interventions_ascii.txt','w',encoding='ascii',errors='ignore') as outfile:
    for line in infile:
        print(*line.split(), file=outfile)

In [110]:
# with open("diseases_ascii.txt", encoding='ascii') as f:
#     diseases_ascii = f.read()
    
# with open("interventions_ascii.txt", encoding='ascii') as f:
#     interventions_ascii = f.read()
    
    
with open("diseases_ascii.txt", encoding='ascii') as f:
    diseases_ascii = f.read().splitlines() 
# print(content_list)

In [111]:
diseases_ascii

["b'schizophreniform disorder'",
 "b'high fat meal - adjusted insulin dose +30 percent'",
 "b'non nutritive sweeteners consumption'",
 "b'venous occlusion'",
 "b'drug dependence '",
 "b'lumbar spine fusion'",
 "b'cystic fibrosis pulmonary'",
 "b'deep neck space infections'",
 "b'mature b-cell lymphoma'",
 "b'high grade squamous intra-epithelial lesion (hsil)'",
 "b'non st segment elevation mi and unstable angina'",
 "b'strabismic amblyopia'",
 "b'body mass index quantitative trait locus 7 disorder'",
 "b'growth retardation'",
 "b'sedation during'",
 "b'systolic and diastolic blood pressure levels'",
 "b'muscle diseases'",
 "b'advanced or metastatic esophageal squamous cell cancer without previous systemic chemotherapy'",
 "b'rhytides'",
 "b'major neurocognitive disorder with aggressive behavior'",
 "b'healthy runners compression stocking and performance'",
 "b'glycopyrrolate'",
 "b' beta -thalassemia major'",
 "b'cell phone use'",
 "b'chronic physical health conditions'",
 "b'plasmodiu

In [115]:
def run_metamap(input_term):
    concepts_dict = dict()
    try:
        concepts,error = mm.extract_concepts([input_term])
        concepts_dict[input_term] = concepts
    except:
        concepts_dict[input_term] = None 
    return(concepts_dict)

In [116]:
number_of_processes = 4
# metamapped_diseases = multiprocessing.Pool(number_of_processes).map(run_metamap, diseases_translated.values())
metamapped_diseases = multiprocessing.Pool(number_of_processes).map(run_metamap, diseases_ascii)
# metamapped_interventions = multiprocessing.Pool(number_of_processes).map(run_metamap, interventions_ascii[:50])


In [140]:
metamapped_diseases

[{"b'schizophreniform disorder'": [ConceptMMI(index='USER', mm='MMI', score='9.99', preferred_name='Schizophreniform Disorders', cui='C0036358', semtypes='[mobd]', trigger='["Schizophreniform Disorder"-tx-1-"schizophreniform disorder"-noun-0]', location='TX', pos_info='6/25', tree_codes='F03.700.675')]},
 {"b'high fat meal - adjusted insulin dose +30 percent'": [ConceptMMI(index='USER', mm='MMI', score='22.38', preferred_name='Insulin', cui='C0021641', semtypes='[aapp,horm,phsu]', trigger='["INSULIN"-tx-1-"insulin"-noun-0]', location='TX', pos_info='31/7', tree_codes='D06.472.699.587.200.500.625;D12.644.548.586.200.500.625'),
   ConceptMMI(index='USER', mm='MMI', score='12.94', preferred_name='Hyperinsulinism', cui='C0020459', semtypes='[dsyn]', trigger='["high insulin"-tx-1-"high insulin"-noun-0]', location='TX', pos_info='6/4,31/7', tree_codes='C18.452.394.968'),
   ConceptMMI(index='USER', mm='MMI', score='12.91', preferred_name='Meal (occasion for eating)', cui='C1998602', semtypes

dict_keys(["b'schizophreniform disorder'"])

In [120]:
with open("metamapped_interventions.txt", 'w') as f: 
    for value in metamapped_interventions: 
        f.write('%s\n' % (value))

In [122]:
type(metamapped_diseases[0])

dict

In [ ]:
"My name is {fname}, I'm {age}".format(fname = "John", age = 36)

In [ ]:
# link nct ids of studies that have the same disease and intervention pair---multiple studies may have studies the same intervention
# restrict returned CUIs of metamapped concepts???

In [ ]:
# final_df['col1'].map(di).fillna(final_df['col1'])
final_df['processed_interventions'] = final_df['object_name'].map(interventions_mappings).fillna(final_df['object_name'])


In [ ]:
final_df

In [ ]:
orig_and_process_interventions = pd.DataFrame(list(zip(interventions, result)))
orig_and_process_interventions.columns = ['input_intervention', 'processed_intervention']

In [ ]:
orig_and_process_interventions

In [ ]:
orig_and_process_interventions.to_csv('processed_interventions_check.csv', sep ='\t', index=False)



In [ ]:
# loop through list of interventions, separate the intervention string into char, use dictionary lookup to pick out non-ascii char and replace in string with value from dict    
processed_interventions = [''.join([ascii_char_dict.get(item,item) for item in list(interventions)]) for intervention in interventions]


In [ ]:
# encode any remaining non-ascii characters
interventions_deascii = [intervention.encode('ascii', 'ignore').decode('ascii') for intervention in processed_interventions]
   
orig_and_process_interventions = pd.DataFrame(list(zip(interventions, interventions_deascii)))
orig_and_process_interventions.columns = ['input_intervention', 'processed_intervention']
orig_and_process_interventions[:100]


In [ ]:
# interventions = final_df['object_name'].unique()
# print(type(interventions))
# len(interventions)


In [ ]:
print(interventions[131405])

In [ ]:
print("how many unique interventions: " + str(len(interventions)))
print("how many unique diseases: " + str(len(diseases)))

In [ ]:
###### COMMENT TO REVERT TO ORIGINAL SIZED DATA SETS, THESE CELLS CREATE SUBSETS OF DATA FOR TESTING PURPOSES

orig_and_process_diseases = orig_and_process_diseases[:200]
orig_and_process_interventions = orig_and_process_interventions[:200]

In [ ]:
with open('processed_diseases_check.csv','w') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['original_disease','disease_de_ascii'])
    for row in orig_and_process_diseases:
        csv_out.writerow(row)


In [ ]:
# testing metamap on small subsets
diseases_mapped = []

for disease in diseases_deascii[6150:6250]:
    disease = [disease]
    concepts,error = mm.extract_concepts(disease)
#     disease_concepts.append(concepts)
    disease_mapped = [disease, concepts]
    diseases_mapped.append(disease_mapped)

mapped_diseases_df = pd.DataFrame(diseases_mapped)  
mapped_diseases_df

In [ ]:
def run_metamap(orig_and_process_diseases_slice):
    mapped_slice = [] 
    for index, row in orig_and_process_diseases_slice.iterrows():
        disease = [row['processed_disease']]  
        concepts,error = mm.extract_concepts(disease)
        mapped_row = [row['input_disease'], row['processed_disease'], concepts]
        mapped_slice.append(mapped_row)
    return(mapped_slice)
        

mappings = pd.DataFrame(columns=['input_disease', 'processed_disease', 'metamapped_disease'])
for i in range(0, len(orig_and_process_diseases), 100):
# for i in range(0, 200, 10):
    mapped = run_metamap(orig_and_process_diseases[i:i+100])
    mapped = pd.DataFrame(mapped, columns=['input_disease', 'processed_disease', 'metamapped_disease'])
    mappings = mappings.append(mapped)
    print("index at: " + str(i))
  


In [ ]:
mappings.to_pickle('mappings.pkl')


In [ ]:
mappings 


In [ ]:
print(len(mappings))

In [ ]:
# sample_mappings = mappings

In [ ]:
sample_mappings

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
mapped

In [ ]:
# use pooling to multiprocess

import psutil 
print(psutil.cpu_count(logical = False))
print(psutil.cpu_count(logical = True))


In [ ]:
# create batch functions to run

In [ ]:
import multiprocessing

process_list = []
for i in range(2):
    p =  multiprocessing.Process(target=run_metamap)
    p.start()
    process_list.append(p)

for process in process_list:
    process.join()

In [ ]:
# create function to run metamap faster and allow pooling
def run_metamap(orig_and_process_diseases_slice):
    mapped_slice = [] 
    for index, row in orig_and_process_diseases_slice.iterrows():
        disease = [row['processed_disease']]  
        concepts,error = mm.extract_concepts(disease)
        mapped_row = [row['input_disease'], row['processed_disease'], concepts]
        mapped_slice.append(mapped_row)
    return(mapped_slice)
        
def batch_create(orig_and_process_diseases):
    mappings = pd.DataFrame(columns=['input_disease', 'processed_disease', 'metamapped_disease'])
    for i in range(0, len(orig_and_process_diseases), 100):
    # for i in range(0, 200, 10):
        mapped = run_metamap(orig_and_process_diseases[i:i+100])
        mapped = pd.DataFrame(mapped, columns=['input_disease', 'processed_disease', 'metamapped_disease'])
        mappings = mappings.append(mapped)
        print("index at: " + str(i))

In [ ]:
batch_create(orig_and_process_diseases)

In [ ]:

process_list = []
for i in range(2):
    p =  multiprocessing.Process(target=batch_create(orig_and_process_diseases))
    p.start()
    process_list.append(p)

for process in process_list:
    process.join()

In [ ]:
target1 = batch_create(orig_and_process_diseases) 
target2 = batch_create(orig_and_process_interventions) 

In [ ]:
# diseases_mappings = dict((i, run_metamap(i)) for i in diseases[:5])
diseases_mappings = {k:run_metamap(v) for k, v in diseases_translated.items()}
diseases_mappings
interventions_mappings = {k:run_metamap(v) for k, v in interventions_translated.items()}
interventions_mappings

In [ ]:
my_list = [i for i in range(10123)]
for i in range(0, len(my_list), 100)
    process_list(my_list[i:i+100])


def process_list(my_list)
    url = 'https://api.example.com'
    data = {'update_list': my_list}
    headers = {'auth': auth}
    r = requests.put(url, data=json.dumps(data), headers=headers)

In [ ]:
# DON'T FORGET TO CHANGE PYMASTER SCRIPT TO OUTPUT SNOMED ONLY TERMS